In [3]:
import pandas as pd
import numpy as np
fake = pd.read_csv("Fake.csv")
true = pd.read_csv("True.csv")
fake_texts = fake["text"].to_numpy()
true_texts = true["text"].to_numpy()

all_texts = np.append(fake_texts, values=true_texts)

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


# Setup Vectorizer

In [4]:
lowercase = False
tf_idf = True

# Run Vectorization

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

if tf_idf:
    vectorizer = TfidfVectorizer(stop_words="english")
else:
    vectorizer = CountVectorizer(analyzer="word", stop_words="english", lowercase=lowercase)

all_tokens = vectorizer.fit_transform(all_texts)
all_tokens.shape

(44898, 121690)

# Create Labels

In [6]:
y_fake = np.linspace(start=0, stop=0, num=fake_texts.size)
y_true = np.linspace(start=1, stop=1, num=true_texts.size)
y = np.append(y_fake, values= y_true)

# Split Labels

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(all_tokens, y, test_size=0.25, random_state=42)

# Store Results

In [8]:
import scipy.sparse

scipy.sparse.save_npz("X_train.npz", X_train)
scipy.sparse.save_npz("X_test", X_test)
np.save("y_train", y_train)
np.save("y_test", y_test)